# CLIPtionary Attack Notebook
<h2>License</h2>
Copyright (c) 2021 Bearsharktopusdev

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# HOW TO USE
<h2>Upload your image in the file thing. Put the image path in Parameters as indicated. Fill in other parameters as desired. Restart + Run All and then go get some coffee because it'll take a little bit.</h2>

# Imports & Shit

In [1]:
#@markdown
!nvidia-smi

Sun Aug 29 19:12:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@markdown
%cd /content/

!git clone https://github.com/openai/CLIP.git

%cd /content/CLIP/

!pip install ftfy
!pip install tqdm
from tqdm import trange

import imageio
import torchvision

from IPython import display
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import clip
!pip install kornia
import kornia
import torch
import torch.nn.functional as F

from PIL import Image

from pprint import pprint

import random

clip.available_models()

#RN50x4
#ViT-B/32

import numpy as np

# Load the model
perceptor, preprocess = clip.load('ViT-B/32', jit=True) # usually use vit
perceptor = perceptor.eval()

clear_output()

In [3]:
#@markdown

%cd /content/
!git clone https://github.com/first20hours/google-10000-english

clear_output()

In [4]:
#@markdown
file = open("/content/google-10000-english/google-10000-english-usa-no-swears-short.txt", "r")
short_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/google-10000-english-usa-no-swears-medium.txt", "r")
medium_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/google-10000-english-usa-no-swears-long.txt", "r")
long_list = file.read().splitlines()
file.close()

file = open("/content/google-10000-english/20k.txt", "r")
full_list = file.read().splitlines()
file.close()

# Parameters

In [5]:
check_amount = 200 #100, number of terms to contest per epoch
seed_epochs = 4000 #10, number of iterations of the checking algorithm for the initial seeding
self_comp_epochs = 5 #4, number of iterations of the self-comparative attack
culling_threshold = 1.0 #1.2, any phrase worth less than this many words is culled from the list. 
# for example, with a threshold of 1 and 100 words, any word that scores less than 0.01 (1/100) is culled
# Recommended to be kept between 1 and 2

seed_words = "sentinel, statue" #"", comma separated list of words to seed the initial epochs with.
image = "/content/vacation.jpg" #don't delete /content/, just upload to google colab and put your filename there

dictionary_from_top = 20 #Adds this many words from the top of the frequency-rated dictionary to the dictionary attack
dictionary_from_random = 20 #Adds this many words randomly from the frequency-rated dictionary to the dictionary attack

iterative_guess_words = "" #"", comma separated list of words for the iterative guess & check step


heavy_dictionary = True #False, when true, runs through a text file of ~500,000 english words with no frequency rating as opposed to the 20,000 word corpus used.
second_run = True #False, when true, runs the initial dictionary seeding a second time just to check for stragglers.
progress_bar_seed = False #False, enables progress bar on the seeding round rather than seeing live results.

In [6]:
#@markdown

if heavy_dictionary:
  !git clone https://github.com/dwyl/english-words
  file = open("/content/english-words/words.txt", "r")
  heavy_list = file.read().splitlines()
  file.close()

clear_output()

# Definitions

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/16", device=device)

image = preprocess(Image.open(image)).unsqueeze(0).to(device)
image_features = model.encode_image(image)

winner_list = []

def rate_words(input_list):
  text = clip.tokenize(input_list).to(device)

  for i in input_list:
    " ".join(i.split())

  with torch.no_grad():
      text_features = model.encode_text(text)
      
      logits_per_image, logits_per_text = model(image, text)
      probs = logits_per_image.softmax(dim=-1).cpu().numpy()

  probs_list = probs.tolist()[0]
  input_probs_zip = zip(input_list, probs_list)
  input_probs_list = []
  for i in input_probs_zip:
    input_probs_list.append(i)
  result = [i for i in input_probs_list if i[1] >= max(culling_threshold/len(input_list), 0.5/check_amount)]
  result = remove_duplicates(result)
  result.sort(key=lambda x:x[1], reverse=True)

  return result

list_index = check_amount
def refill_list(input_list):
  global list_index
  pull_list = []
  if heavy_dictionary:
    pull_list = heavy_list
  else:
    pull_list = full_list
  while len(input_list) < check_amount and list_index < len(pull_list):
    input_list.append(pull_list[list_index])
    list_index = list_index + 1
  if list_index >= len(pull_list):
    random_list = random.sample(pull_list, check_amount)
    while len(input_list) < check_amount:
      input_list.append(random_list[0])
      random_list.pop(0)

def remove_duplicates(input_list):
    return list(set([i for i in input_list]))

def attack(input_list):
  for k in input_list:
    global winner_list
    winner_list = remove_duplicates(winner_list)
    thing = [x[0] for x in winner_list]
    thinglen = len(thing)
    thinglenlen = len(str.split(thing[0]))+1
    new_list = [thing]*thinglenlen
    new_new_list = []
    for i in range(thinglenlen):
      for j in new_list[i]:
        newj = str.split(j)
        newj.insert(i, k)
        newj = " ".join(newj)
        new_new_list.append(newj)
    for i in thing:
      new_new_list.append(i)
    new_new_list.append(k)

    winner_list = rate_words(new_new_list)

    print('\n---\nprinting top 10')
    winner_list.sort(key=lambda x:x[1], reverse=True)
    for i in range(len(winner_list)):
      if i < 10:
        print(winner_list[i])

100%|███████████████████████████████████████| 335M/335M [00:05<00:00, 61.9MiB/s]


# Dictionary Seeding

In [ ]:
seed_word_list = seed_words.split(", ")
if heavy_dictionary:
  initial_attack = heavy_list[0:check_amount - len(seed_word_list)]
else:
  initial_attack = full_list[0:check_amount - len(seed_word_list)]
initial_attack = initial_attack + seed_word_list

if progress_bar_seed:
  for i in trange(seed_epochs):
    refill_list(initial_attack)
    winner_list = rate_words(initial_attack)
    initial_attack = [x[0] for x in winner_list]

  print('\n---')
  for i in winner_list:
    print(i)
else:
  for i in range(seed_epochs):
    refill_list(initial_attack)
    winner_list = rate_words(initial_attack)
    initial_attack = [x[0] for x in winner_list]

    display.clear_output(wait=True)
    pprint(winner_list[0:20])

# Self-Comparing Attack

In [ ]:
self_comp_attack = []
temp = []

new = [x[0] for x in winner_list]

for j in range(self_comp_epochs):
  if j == 0:
    for k in new:
      for i in new:
        self_comp_attack.append(k + " " + i)
      self_comp_attack.append(k)
  else:
    for k in new:
      for i in temp:
        self_comp_attack.append(k + " " + i)

  winner_list = rate_words(self_comp_attack)
  self_comp_attack = [x[0] for x in winner_list]
  temp = [x[0] for x in winner_list]

  print('\n---\nprinting top 10')
  for i in range(len(winner_list)):
    if i < 10:
      print(winner_list[i])

# Iterative Dictionary Attack

In [ ]:
dictionary_attack = full_list[0:dictionary_from_top]
random_list = random.sample(full_list, dictionary_from_random)
dictionary_attack = random_list + dictionary_attack

attack(dictionary_attack)

# Synonym/Antonym Attack

# Iterative Guess & Check

In [ ]:
words_list = iterative_guess_words.split(", ")

attack(words_list)